### Guide to the notebook 

__DFs involved here:__ 
- __Train:__ The basic training set with 4 features
- __FullTrain:__ Complete set of transactions made by the customers in the training set.

__The notebook in a nutshell:__ All that is happening here is two primary steps: First I make an aggregation using FullTrain (set of transactions). Then add the aggregated column to Train (the basic training set). This would create an aggregated training set to which I will add aggregated merchant data later.

### Importing Libs & Data

In [1]:
import pandas as pd
import numpy as np

The cell below just imports the relevant DFs. Drops the training set features from the FullTrain DF as they are redundant. A copy of transactions DF is created. All transactions that were not approved are dropped (this improved results).

In [2]:
%%time 
Train=pd.read_csv('train.csv'); FullTrain=pd.read_pickle('FullTrain')

FullTrainAgg=FullTrain.copy()

FullTrainAgg.drop(['first_active_month','feature_1','feature_2','feature_3','target'],axis=1,inplace=True)

FullTrainAgg = FullTrainAgg[FullTrainAgg.authorized_flag != 'N']

CPU times: user 2.66 s, sys: 1.76 s, total: 4.42 s
Wall time: 18.9 s


__Plan to aggregate__

1. Use the FullTrain df to apply the aggregations decided so far
2. Create a copy of FullTrain and start on that

### 1. Total transactions found and merged with Train, so Train is now updated

I've added comments to the cell below, to explain the process. This has only been done for two instances. Since the steps in all aggregations are identical, its redundant to explain for all instances.

In [10]:
FullTrainAgg['card_id_2'] = FullTrainAgg['card_id'] # Creating an extra column for counting

TotTransPerCard = FullTrainAgg[['card_id','card_id_2']] # Creating New DF containing only the feature to be aggregated

TotTransPerCard = TotTransPerCard.groupby((['card_id'])).count() # Necessary aggregation performed

Train=pd.merge(Train,TotTransPerCard,how='left',on='card_id') # Aggregated feature added to training set for later use

Train=Train.rename({'card_id_2':'TransTotNum'},axis=1);Train.head() # New feature renamed for clarity

,first_active_month,card_id,feature_1,feature_2,feature_3,target,TransTotNum
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,275
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,360
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,43
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,96
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,173


### 2. Total Purchases, combined with Train as well

In [11]:
# Aggregating by purchase amounts of each card_id

TotPurAmt = FullTrainAgg[['card_id','purchase_amount']] # Creating New DF containing only the feature to be aggregated

TotPurAmt = TotPurAmt.groupby((['card_id'])).sum() # Performing necessary aggregation

Train=pd.merge(Train,TotPurAmt,how='left',on='card_id') # Merging with training set for later use

,first_active_month,card_id,feature_1,feature_2,feature_3,target,TransTotNum,purchase_amount
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,275,-174.081299
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,360,-221.010391
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,43,-29.239578
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,96,-62.278957
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,173,-92.751595
5,2016-09,C_ID_0894217f2f,4,2,0,0.871585,36,-14.564627
6,2016-12,C_ID_7e63323c00,3,2,1,0.230129,249,-177.439667
7,2017-09,C_ID_dfa21fc124,3,2,1,2.135850,27,-17.345419
8,2017-08,C_ID_fe0fdac8ea,2,1,0,-0.065406,13,-6.986907
9,2016-08,C_ID_bf62c0b49d,2,2,0,0.300062,106,-72.413033


### 3. Aggregating approved Trans

In [87]:
# Aggregating by approved purchases of each card_id

ApprTrans = FullTrainAgg[['card_id','authorized_flag']]

ApprTrans = ApprTrans.groupby((['card_id'])).agg({'authorized_flag' : lambda x: (x=='Y').sum()})

Train=pd.merge(Train,ApprTrans,how='left',on='card_id');Train

Train.rename(mapper={'authorized_flag':'NumApprTrans'},axis=1,inplace=True)

### 4. Total of each category in Trans table

In [43]:
FullTrainAgg.category_3.value_counts()

A    10571500
B     8007626
C     1301271
Name: category_3, dtype: int64

In [12]:
# category_1_x. Only totalling Y since it has only 2 cats

Cat1Y = FullTrainAgg[['card_id','category_1_x']]

Cat1Y = Cat1Y.groupby((['card_id'])).agg({'category_1_x' : lambda x: (x=='Y').sum()})

Train=pd.merge(Train,Cat1Y,how='left',on='card_id');Train

Train.rename(mapper={'category_1_x':'Cat_1_Y'},axis=1,inplace=True);Train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,TransTotNum,purchase_amount,Cat_1_Y
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,275,-174.081299,0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,360,-221.010391,33
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,43,-29.239578,0
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,96,-62.278957,14
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,173,-92.751595,15
5,2016-09,C_ID_0894217f2f,4,2,0,0.871585,36,-14.564627,1
6,2016-12,C_ID_7e63323c00,3,2,1,0.230129,249,-177.439667,0
7,2017-09,C_ID_dfa21fc124,3,2,1,2.135850,27,-17.345419,0
8,2017-08,C_ID_fe0fdac8ea,2,1,0,-0.065406,13,-6.986907,0
9,2016-08,C_ID_bf62c0b49d,2,2,0,0.300062,106,-72.413033,0


In [13]:
# Category 2 has 5 values so taking 1 to 4
# Create seperate columns for ease of aggregation later

FullTrainAgg['category_1_1']=FullTrainAgg['category_2_x']; FullTrainAgg['category_1_2']=FullTrainAgg['category_2_x']; FullTrainAgg['category_1_3']=FullTrainAgg['category_2_x']; FullTrainAgg['category_1_4']=FullTrainAgg['category_2_x'] 

Cat2Counts = FullTrainAgg[['card_id','category_1_1','category_1_2','category_1_3','category_1_4']]

Cat2Counts = Cat2Counts.groupby((['card_id'])).agg({'category_1_1' : lambda x: (x.eq('1.00000000')).sum(),\
                                                    'category_1_2' : lambda x: (x.eq('2.00000000')).sum(),\
                                                   'category_1_3' : lambda x: (x.eq('3.00000000')).sum(),\
                                                   'category_1_4' : lambda x: (x.eq('4.00000000')).sum()})

Train=pd.merge(Train, Cat2Counts, how='left', on='card_id')

Train.rename(mapper={'category_1_1':'Cat2Tot1','category_1_2':'Cat2Tot2','category_1_3':'Cat2Tot3','category_1_4':'Cat2Tot4'},\
            axis=1, inplace=True)

In [14]:
# Category 3 has A,B and C so doing totals of A & B

FullTrainAgg['category_3_A']=FullTrainAgg['category_3']; FullTrainAgg['category_3_B']=FullTrainAgg['category_3']

Cat2Counts = FullTrainAgg[['card_id','category_3_A','category_3_B']]

Cat2Counts = Cat2Counts.groupby((['card_id'])).agg({'category_3_A' : lambda x: (x=='A').sum(),\
                                                    'category_3_B' : lambda x: (x=='B').sum()})

Train=pd.merge(Train, Cat2Counts, how='left', on='card_id')

Train.rename(mapper={'category_3_A':'Cat3TotA', 'category_3_B':'Cat3TotB'}, axis=1, inplace=True)

### 4. Total # of installments for each card_id

In [55]:
FullTrainAgg.installments.describe()

count    2.002573e+07
mean     6.514931e-01
std      2.633509e+00
min     -1.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.000000e+00
max      9.990000e+02
Name: installments, dtype: float64

In [15]:
# Aggregating by total # of installments for each card_id

TotInsNum = FullTrainAgg[['card_id','installments']]

TotInsNum = TotInsNum.groupby((['card_id'])).sum()

Train=pd.merge(Train,TotInsNum,how='left',on='card_id');

Train.rename(mapper={'installments':'TotInsNum'}, axis=1, inplace=True);Train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,TransTotNum,purchase_amount,Cat_1_Y,Cat2Tot1,Cat2Tot2,Cat2Tot3,Cat2Tot4,Cat3TotA,Cat3TotB,TotInsNum
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,275,-174.081299,0,272,0,0,0,275,0,0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,360,-221.010391,33,327,0,0,0,0,286,554
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,43,-29.239578,0,5,0,0,0,43,0,0
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,96,-62.278957,14,20,0,0,62,0,85,103
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,173,-92.751595,15,19,0,7,131,0,167,188


### 5. Avg Purchase amount

In [17]:
# Find using existing columns: TotPurAmt / NumApprTrans

Train['AvgPurAmt'] = Train['purchase_amount'] / Train['TransTotNum'];Train.AvgPurAmt.describe()

count    201917.000000
mean         -0.540173
std           0.367334
min          -0.745371
25%          -0.679621
50%          -0.626560
75%          -0.521473
max          36.454815
Name: AvgPurAmt, dtype: float64

### 6. Latest transaction for each card_id

In [18]:
# This part only removes the timestamp from purchase_date

FullTrainAgg.purchase_date=pd.to_datetime(FullTrainAgg.purchase_date,format='%Y-%m-%d')

FullTrainAgg['purchase_date']=FullTrainAgg['purchase_date'].dt.date

FullTrainAgg.purchase_date.max() # Its finding the correct max

LatestPurDate = FullTrainAgg[['card_id','purchase_date']]

LatestPurDate = LatestPurDate.groupby((['card_id'])).max()

Train=pd.merge(Train, LatestPurDate, how='left',on='card_id')

Train.rename(mapper={'purchase_date':'LatestPurDate'},axis=1,inplace=True)

### 7. Length of time from first active month to latest purchase

In [19]:
Train.first_active_month = pd.to_datetime(Train.first_active_month)

Train.first_active_month = Train.first_active_month.dt.date

Train.first_active_month.describe()

# Use existing columns: LatestPurDate - first_active_month

Train['TimeSpent'] = Train['LatestPurDate'] - Train['first_active_month']

### 8. Purchase amount over time spent - CLV proxy

In [20]:
# Use existing columns: TotPurAmt - TimeSpent

Train['CLV'] = Train['purchase_amount'] / Train['TimeSpent'].dt.days; Train.CLV.head(10)

0   -0.524341
1   -0.487882
2   -0.046047
3   -0.271961
4   -0.521076
5   -0.038429
6   -0.361384
7   -0.102032
8   -0.030246
9   -0.117745
Name: CLV, dtype: float32

In [107]:
Train.to_pickle('TrainAgg2')